In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/churn-prediction-of-bank-customers/Churn_Modelling.csv')

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
sns.set_style('white')
plt.style.use('tableau-colorblind10')
#plt.xkcd()

In [ ]:
plt.style.available

In [ ]:
ax = sns.countplot('Exited',data=data) #Imbalanced Dataset

In [ ]:
ax = sns.countplot('Gender',data=data)

In [ ]:
ax = sns.countplot('NumOfProducts',data=data)

In [ ]:
ax = sns.boxplot(x='Exited',y='EstimatedSalary',data=data)

In [ ]:
ax = sns.boxplot(x='NumOfProducts',y='Age',hue='Exited',data=data)

In [ ]:
ax = sns.boxplot(x='Gender',y='Tenure',data=data)

In [ ]:
ax = sns.boxplot(x='Gender',y='Tenure',hue='Exited',data=data)

In [ ]:
#HasCrCard
ax = sns.countplot('HasCrCard',data=data)

In [ ]:
ax = sns.countplot('HasCrCard',hue='Exited',data=data)

In [ ]:
ax = sns.boxplot(x='Exited',y='Age',data=data)

In [ ]:
ax = sns.boxenplot(x='Exited',y='Age',data=data)

In [ ]:
ax = sns.distplot(data.Age)

In [ ]:
#EstimatedSalary
ax = sns.distplot(data.EstimatedSalary)

In [ ]:
data.EstimatedSalary.describe()

In [ ]:
ax = data['EstimatedSalary'].hist()

### Lifelines package for Survival Analysis

In [ ]:
!pip install lifelines

In [ ]:
import lifelines

In [ ]:
dir(lifelines) #print lifelines methods

In [ ]:
T = data.Tenure.values
E = data.Exited.values

In [ ]:
T[:5]

In [ ]:
E[:5]

In [ ]:
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E)  # or, more succinctly, kmf.fit(T, E)

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
kmf.survival_function_
kmf.cumulative_density_
ax = kmf.plot_survival_function() # or just kmf.plot()

In [ ]:
ax = kmf.plot_cumulative_density()

In [ ]:
data.Tenure.describe()

In [ ]:
data.head()

In [ ]:
ax = plt.subplot(111)

kmf = KaplanMeierFitter()

for name, grouped_df in data.groupby('Gender'):
    kmf.fit(grouped_df["Tenure"], grouped_df["Exited"], label=name)
    kmf.plot(ax=ax)

In [ ]:
ax = plt.subplot(111)

kmf = KaplanMeierFitter()

for name, grouped_df in data.groupby('Geography'):
    kmf.fit(grouped_df["Tenure"], grouped_df["Exited"], label=name)
    kmf.plot(ax=ax)

In [ ]:
ax = sns.boxplot('CreditScore',data=data,orient='v')

In [ ]:
ax = sns.boxenplot('CreditScore',data=data)

In [ ]:
ax = sns.distplot(data.CreditScore)

In [ ]:
data['CreditScore_Cat'] = pd.cut(data['CreditScore'],bins=3,labels=['low','mid','high'])

In [ ]:
ax = sns.countplot('CreditScore_Cat',hue='Exited',data=data)

In [ ]:
ax = sns.boxplot(x='Exited',y='CreditScore',data=data)

In [ ]:
data.NumOfProducts.value_counts()

In [ ]:
data.HasCrCard.value_counts()

In [ ]:
ax = sns.countplot('IsActiveMember',hue='Exited',data=data)

In [ ]:
data.IsActiveMember.value_counts()

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
catCols = ['Geography', 'Gender', 'NumOfProducts', 'HasCrCard']

In [ ]:
for col in catCols:
    data[col] = data[col].astype('category')

In [ ]:
data.dtypes

In [ ]:
data_copy = data.copy()

In [ ]:
data_copy = data_copy[['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'EstimatedSalary', 'Exited']]

In [ ]:
# Creating dummy variables for categorical datatypes
data_filtered = pd.get_dummies(data_copy, columns=catCols)

In [ ]:
data_filtered.head()

In [ ]:
data_final.columns

In [ ]:
#data_final["Exited"] = data_final.Exited.astype('int')

In [ ]:
data_filtered.columns

In [ ]:
data_final.dtypes

In [ ]:
data_filtered.columns

In [ ]:
data_final = data_filtered[['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'Exited']]

In [ ]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(data_filtered)

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(data_final, duration_col='Tenure', event_col='Exited')

cph.print_summary()  # access the individual results using cph.summary

### 1 unit increase in Age will increase baseline hazard by about 4%

In [ ]:
ax = cph.plot()

In [ ]:
#from lifelines.calibration import survival_probability_calibration
#survival_probability_calibration(cph, data_final, t0=25)

In [ ]:
#Predictions
cph.predict_survival_function(data_final)

In [ ]:
cph.predict_median(data_final)

In [ ]:
cph.predict_partial_hazard(data_final)

In [ ]:
a = cph.predict_median(data_final)
b = cph.predict_partial_hazard(data_final)

In [ ]:
data.head()

In [ ]:
data[data.Exited==1].head()

In [ ]:
data_preds = data.copy()

In [ ]:
data_preds['median_survival'] = a

In [ ]:
data_preds['partial_hazard'] = b

In [ ]:
data_preds.head()

In [ ]:
data_preds.median_survival.value_counts()

In [ ]:
ax = sns.distplot(data_preds.partial_hazard)

In [ ]:
##Still in progress